<div style='width=100%; display:flex;flex-direction:row'><img  src=https://universidadeuropea.com/resources/media/images/universidad-europea-logo_poc9mEM.original.png width=100  style='  margin-left: auto;margin-right: auto; width: 25%; height:25%;'><img  src=https://i.ibb.co/1068C7j/EATEASER.jpg width=100 style='  margin-left: auto;margin-right: auto; width: 10%;height:25%;'></div>

<div style='margin:auto;text-align: center;font-family: "Times New Roman", Times, serif; font-weight: bold;'>PROYECTO COMUTACIONAL<br><br>EATEASER - VOZ A TEXTO</div>

<div style='width:100%; display:flex;flex-direction:row'>
    <div style='width:50%;margin-right:5cm;'>
        <p style='font-family: "Times New Roman", Times, serif; font-weight: bold;'>ESTUDIANTES</p>
<ul style='font-family: "Times New Roman", Times, serif;'>
    <li>Adilem Dobras 21911633</li><li>Roberto Echevarria 21823680</li><li>Carlos Gonzales 22067726</li><li>Juan Carlos Rondeau 21816176</li></ul> </div>

</div>

<h2 style="font-size: 19px;color:#6DA0FF;font-family:Georgia, Times, 'Times New Roman', serif;letter-spacing: 3px;font-weight: normal">1. Importamos las librerias</h2>

In [1]:
import glob
import os
import re
import requests
import pathlib
import sys

try:
    from pytube import YouTube
    from pytube import Playlist
except ModuleNotFoundError:
    !pip install pytube
try:
    import speech_recognition as sr
except ModuleNotFoundError:
    !pip install SpeechRecognition
try:
    from pydub import AudioSegment
    from pydub.silence import split_on_silence
except:
    !pip install pydub
try:
    import moviepy.editor as mp
except:
    !pip install moviepy
try:
    from bs4 import BeautifulSoup
except:
    !pip install beautifulsoup4
try:
    from nltk.stem import PorterStemmer
    from nltk.tokenize import word_tokenize
except:
    !pip install nltk  

<h2 style="font-size: 19px;color:#6DA0FF;font-family:Georgia, Times, 'Times New Roman', serif;letter-spacing: 3px;font-weight: normal">2. Inicio del programa</h2>

<h3  style='font-family: "Times New Roman", Times, serif; font-weight: bold;text-align:center;font-size:14px'>CLASE CONTROLADORVIDEO</h3><p style='font-family: "Times New Roman", Times, serif; font-size:14px'>En esta clase se realizará los ajustes para manejar el video recibido y manipularlo.</p>


In [2]:
class ControladorVideo:
    def __init__(self,enlace): 
        self._idvideo = self.reenumerar()
        self.enlacevideo=enlace
        self.yt=YouTube(self.enlacevideo)
        self.nombrevideo=''
        self.titulovideo=self.yt.title
        self.autorvideo=self.yt.author
        self.fechavideo=self.yt.publish_date
        self.duracionvideo=self.yt.length
    """|DESCARGAR VIDEO URL: descarga el video de youtube
       |return: devuelve una ruta absoluta"""
    def descargarVideoURL(self):
        recetasVideos = 'recetasvideos/'
        #aqui creo un nuevo id para el nuevo video
        self._idvideo= self._idvideo+1
        #esta sera el archivo del video y su nuevo nombre
        nombre='receta'+str(self._idvideo)
        #le pedimos al pytube que solo nos descargue el audio y lo descargamos
        t=self.yt.streams.filter(file_extension='mp4').first().download(output_path=recetasVideos,filename=nombre+'.mp4')
        #devolvemos el nombre
        return nombre
    """|PARSEO VIDEO: pasa el video de .mp4 a .wav
       |nombre: es un string que se colocara el nombre del video
       |return: devuelve el nuevo nombre del audio en .wav"""
    def parseoVideo(self,nombre):
        recetasVideos = 'recetasvideos/'
        #tomamos el video en mp4 
        track = mp.VideoFileClip(recetasVideos+nombre+'.mp4')
        #cambiamos el video a .wav
        nombre_wav="{}.wav".format(nombre)
        track.audio.write_audiofile(recetasVideos+nombre_wav)
        track.close()
        return nombre+'.wav'
    """|REENUMERAR: Devuelve cual es el id del siguiente video.
       |primer for: recorre cada carpeta de /recetastextos
       |segundo for: cuenta cada archivo dentro de /recetastextos/* """
    def reenumerar(self):
        contador = 0
        for item in pathlib.Path("./recetastextos/").iterdir():
            if item.is_file():
                contador += 1
            elif item.is_dir():
                for subitem in item.iterdir():
                    if subitem.is_file():                     
                        contador += 1
        return contador
    """|SPEECH TEXT:Transforma el audio a texto
       |nombre: es un string que se colocara el nombre del video
       |return: devuelve un string con el texto devuelto"""
    def speech_text(self,nombre):
        recetasVideos = 'recetasvideos/'
        #instanciamos el recognizer
        r = sr.Recognizer()
        audio = sr.AudioFile(recetasVideos+nombre)
        with audio as source:
            audio_file = r.record(source)
        #transcribimos el audio a texto
        result = r.recognize_google(audio_file, language = 'es-ES')
        return result
        

<h3  style='font-family: "Times New Roman", Times, serif; font-weight: bold;text-align:center;font-size:14px'>CLASE DEPURADOR</h3><p style='font-family: "Times New Roman", Times, serif; font-size:14px'>En esta clase se realizará el proceso de extraccion, transformacion y carga de nuestro programa EATEASER.</p>


In [3]:
#si el video es mayor de 3 minutos no funciona
#si el video es en ingles no funciona
class Depurador:
    def __init__(self): 
        self.rec=RecursosAdicionales()
    """|VIDEO: proceso etl donde extraemos al informacion del video 
       |enlace: es un string que se colocara el enlace del video"""
    def video(self,enlace):
        try:
            #instanciamos el controlador de videos
            cv=ControladorVideo(enlace)
            #paso 1: descargamos el video
            cv.nombrevideo=cv.descargarVideoURL()
            #paso 2: pasamos el video a .wav
            nombre=cv.parseoVideo(cv.nombrevideo)
            #paso 3: pasamos el .wav a texto
            try:
                result=cv.speech_text(nombre)
                #paso 4: escribimos el texto recibido en un txt
                try:
                    quitarEmojis = dict.fromkeys(range(0x10000, sys.maxunicode + 1), 'NULL')
                    tituloSinEmojis=cv.titulovideo.translate(quitarEmojis)
                    autorSinEmojis=cv.autorvideo.translate(quitarEmojis)
                    resultado=self.rec.escritura(cv.nombrevideo,"Titulo:"+tituloSinEmojis+"\n"+"Autor:"+autorSinEmojis+"\n"+"Fecha Publicacion:"+str(cv.fechavideo)+"\n"+"Entradilla:"+result)
                except:
                    print("fallo al erscribr el video en texto")
                    #paso 5: eliminamos los mp4
                    self.rec.eliminacionMp4()
                    return resultado   
            except:
                print("No se ha podido transcribir el video: "+ cv.nombrevideo + " - "+ cv.titulovideo)
                self.rec.eliminacionMp4()
                return None
        except:
            print("No se ha podido descargar el video: "+ cv.nombrevideo + " - "+ cv.titulovideo)
            return None
    def lista(self, enlace):
        playlist_urls = Playlist(enlace)
        for url in playlist_urls:
            self.video(url)
    def transformacion(self):
        print()
    def carga(self):
        print()

In [4]:
#aqui iran lecturas-escrituras-guardar-eliminar cosas en bases de datos
class RecursosAdicionales:
    """|ESCRITURA: escribe textos txt
       |nombre: nombre del 
       |return: devuelve el audio en texto"""
    #yo siento que en vez de cambiar de carpeta los textos...deberiamos eliminar los mp4
    #y si lo mas factible aqui es que guardemos el video por su nombre?
    #creo que no debemos contar los files porque cansan el programa
    
    def escritura(self,nombre,texto):
        recetasTextos = './recetastextos/'
        if not(os.path.exists(recetasTextos)):
            os.mkdir(recetasTextos)
        f = open(recetasTextos+nombre+'.txt', 'w')
        f.write(texto)
        f = open(recetasTextos+nombre+'.txt', "r")
        print(f.read())
        f.close()
    def eliminacionMp4(self):
        recetasVideos = './recetasvideos/'
        py_files = glob.glob(recetasVideos+'*.mp4')
        for py_file in py_files:
            try:
                os.remove(py_file)
            except OSError as e:
                print(f"Error:{ e.strerror}")      
        

In [5]:
class WebScrap:
    def __init__(self): 
        self.headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    def request(self, url):
        request1 = requests.get(url, headers=self.headers)
        html = request1.content
        soup = BeautifulSoup(html, 'html.parser')
        return soup
    def verificar_alimento(self,alimento):
        soup = self.request( 'https://www.themealdb.com/api/json/v1/1/search.php?s='+alimento)
        print(soup)
        

<h2 style="font-size: 19px;color:#6DA0FF;font-family:Georgia, Times, 'Times New Roman', serif;letter-spacing: 3px;font-weight: normal">3. Main</h2>

In [6]:
#y si tambien vemos si le permitimos al usuario que meta videos?
dep=Depurador()
if __name__ == '__main__':
    #dep.video('https://www.youtube.com/watch?v=6PzQY1E2s2g&list=PLxHmjpcgU5ArC2rY5cpoIcZoVKB_0UHfR&ab_channel=PlatosF%C3%A1cilesconTamara')
    #dep.video('https://www.youtube.com/watch?v=PsqR5M8rdjA&list=LL&index=9&t=4s')
    #dep.video('https://www.youtube.com/watch?v=xfYcM_jHgPY')
    #dep.video('https://www.youtube.com/watch?v=wiCfqc5W-yo')
    #error_nuevo#dep.video('https://www.youtube.com/watch?v=3DnPkf9rP_0')
    #error_nuevo#dep.video('https://www.youtube.com/watch?v=xVsgKMZFCZY')
    #dep.video('https://www.youtube.com/watch?v=rpCe0RPMY94')
    #dep.video('https://www.youtube.com/watch?v=rv4gLMa-FYk')
    #dep.video('https://www.youtube.com/watch?v=VS8zYxBj4r8')
    #dep.video('https://www.youtube.com/watch?v=o99JXrEkZoo')
    #dep.video('https://www.youtube.com/watch?v=lKkg5L23b3M')
    #dep.video('https://www.youtube.com/watch?v=PsqR5M8rdjA&t=14s')
    #dep.video('https://www.youtube.com/watch?v=IvZaAL6qYe0&t=29s')
    #dep.video('https://www.youtube.com/watch?v=SIMQBuuyE9M')
    #dep.video('https://www.youtube.com/watch?v=_YoZfg7R8Hk')
    #dep.video('https://www.youtube.com/watch?v=Zv7KdlOBk7Y')
    #dep.video('https://www.youtube.com/watch?v=mFcN4btaZyI&t=2s')
    #dep.video('https://www.youtube.com/watch?v=sRmmQBBln9Q')
    #dep.video('https://www.youtube.com/watch?v=-QoTJJJfeEE')
    #dep.video('https://www.youtube.com/watch?v=JRY5obPKPzo&list=PLxHmjpcgU5ArC2rY5cpoIcZoVKB_0UHfR&index=5&ab_channel=PlatosF%C3%A1cilesconTamara')
    #dep.video('https://www.youtube.com/watch?v=stFmx7OCy1k&ab_channel=RecetasdeEsbieta')
    #error_videomuylargo#dep.video("https://www.youtube.com/watch?v=qqTqePGIjhc")
    #Lista sin descargar haciendo pruebas
    dep.lista("https://www.youtube.com/playlist?list=PL5lLFKePklpDiUHO2U1wvxH2ES2FKH8ks")
    

MoviePy - Writing audio in recetasvideos/receta29.wav


MoviePy - Done.
Titulo:COCINA PARA TODOS · BROWNIE A LA TAZA
Autor:Reclutados
Fecha Publicacion:2014-04-30 00:00:00
Entradilla:hola soy Mary y hoy estoy aquí en reclutado para enseñaros cómo hacer brownie a la taza pero antes me voy a transformar en cocina todo de leche y 3 cucharadas de cacao en polvo y con ayuda de un tenedor y añadimos la leche y el aceite y volvemos a batir con ayuda de continuación añadimos el azúcar la harina y el cacao y con la ayuda el tenedor lo batimos ya lo tenemos bien batido os tiene que quedar con esta consistencia y naves que ya lo tenemos así lo vamos a poner sobre un plato pequeño como este y lo vamos a llevar al microondas durante 2 minutos y medio ya está listo vamos a cerrar y lo ponemos en marcha una vez que lo hemos sacado del microondas vamos a dejarlo reposar durante 2 minutos ya han pasado 2 minutos y está listo para comer así que voy a probarla me gusta espero que os haya gustado la receta que la he sacado de la página azúcar de colores es que

MoviePy - Done.
Titulo:Galletas Chocolate Chip Cookies con Nutella
Autor:Cocina Para Todos
Fecha Publicacion:2015-05-30 00:00:00
Entradilla:hola soy Mary bienvenido cocina para todos vamos a hacer unas galletas cookies de nutella y para hacerla vamos a necesitar los siguientes ingredientes 220 gramos de harina de trigo 115 gramos o media taza de mantequilla 2 huevos 150 gramos son dos tercios de trabajo de azúcar 150 g o una taza de chip de chocolate 115 gramos o media taza de crema de cacao y avellanas como puedes ver Nutella una cucharada de levadura química también conocida como polvos para hornear y una cucharada de esencia de vainilla lo primero que vamos a hacer es poner la mantequilla que está a temperatura ambiente en un recipiente y le añadimos el azúcar y con ayuda de una batidora de varillas batimos bien una vez que tengamos una mezcla blanquecina vamos añadir la esencia y los dos huevos que están a temperatura ambiente y lo volvemos a mezclar bien ahora vamos a pasar a mezc

MoviePy - Done.
Titulo:Tortitas Americanas Fáciles y Esponjosas | Hot Cakes | Pancakes
Autor:Cocina Para Todos
Fecha Publicacion:2016-04-26 00:00:00
Entradilla:hola soy Mari bienvenida otro tiene para todos si me seguís en el Instagram para todos habréis visto que la semana pasada estuve de cruceros por el Mediterráneo y con tanto cambio temperatura pues al final he acabado un poco resfriada y con la voz chunga pero bueno no os preocupéis que esto se solucionará enseguida y bueno lo que iba que en todos los buffet del crucero no había tortitas americanas había bacon adiós huevo frito ayer salchichas de todo menos tortitas americanas así que vente me encargas de prepararme la cuando volviera y vamos a hacerlas todas juntas así que venga come en un recipiente se pone un huevo dos cucharadas de azúcar 2 cucharadas de aceite suave como puede ser de oliva o de girasol un poco más de media taza o vaso de leche una pizca de sal y una taza o 130 gramos de harina de trigo media cucharada de lev

MoviePy - Done.
No se ha podido transcribir el video: receta32 - Cómo Hacer Brownies de Chocolate
MoviePy - Writing audio in recetasvideos/receta32.wav


...

MoviePy - Done.
Titulo:Galletas Chocolate Chip Cookies súper deliciosas!!
Autor:Cocina Para Todos
Fecha Publicacion:2016-02-01 00:00:00
Entradilla:hola soy Miriam venido cocina para todos hoy tengo una noticia muy importante quedar hoy y es que tonto no tonto tonto ton muchísimas gracias a todos sois los mejores de azúcar un cuarto de taza o 55 g de mantequilla un huevo un cuarto de taza o 30 g de cacao en polvo aunque si no tenéis podéis utilizar cacao soluble 70 chocolate en mi caso es negro tres cuartos de taza o 120 gramos de chips de chocolate aunque también puede utilizar chocolate troceado un cuarto de cucharadita de sal un cuarto de cucharadita de bicarbonato sódico o polvos para hornear y media cucharadita de esencia vainilla otra vez liado lo primero que voy a hacer es poner el horno precalentar a 165 grados centígrados a continuación he puesto la mantequilla y el chocolate juntos en el recipiente te he puesto el huevo y lo voy a ir el azúcar con una batidora de varillas lo l

MoviePy - Done.
No se ha podido transcribir el video: receta34 - Galletas Cookies de Chocolate y Nueces
MoviePy - Writing audio in recetasvideos/receta34.wav


MoviePy - Done.
No se ha podido transcribir el video: receta34 - Coulant a los 4 Chocolates | Volcán de Chocolate
MoviePy - Writing audio in recetasvideos/receta34.wav


MoviePy - Done.
Titulo:Rollitos de Nutella y Canela | Nutella French Toast Rolls
Autor:Cocina Para Todos
Fecha Publicacion:2015-03-28 00:00:00
Entradilla:hola soy medio me digas la cocina para todos Hoy vamos a hacer churros rellenos o rollitos de nutella y para hacerlo vamos a necesitar los siguientes ingredientes una rebanada de pan de molde por Caballito en mi caso son seis a las que le he quitado los bordes una cucharadita de crema de cacao y avellanas cómo puede ser Nutella para cada rollito un huevo un chorrito de leche para freírlo cómo se utilizar un poquito de aceite o mantequilla y para rebozarlos azúcar y canela al gusto en mi caso es una cucharadita lo primero que vamos a hacer es prensar el pan de molde yo lo voy a hacer con un rodillo pero podéis hacerlo con un vaso o con la mano y ahora vamos a pasar el tender una cucharadita de crema de cacao y avellanas en cada pan de molde así en esta parte solo pasamos enrollarlo y el viernes con todos unas que tenemos todos los roll

MoviePy - Done.
No se ha podido transcribir el video: receta35 - Arroz tres delicias chino
